### Objective:
Create SBA Centers for San Diego County

In [1]:
from arcgis.gis import GIS
import arcgis.network as network
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.features import Feature, FeatureSet

import psycopg2

import pandas as pd

import sys
sys.path.append('../../')
from utils import get_config, execute_sql, server_connect

sys.path.append('../')
from gis_resources import create_feature_layer

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

Getting SBA Addresses from postgres 

In [3]:
sql = """
    select organization_type, organization_name, address, contact_number from "community-facing_organizations"
"""

In [4]:
conn = server_connect()
results = execute_sql(conn, sql)
#results

In [5]:
sbaDF = pd.DataFrame(results,
              columns=("org_type","org_name", "address", "cont_nmbr")
              );
sbaDF

,org_type,org_name,address,cont_nmbr
0,Small Business Development Center,San Diego and Imperial SBDC Network (Lead Office),880 National City Blvd,619-482-6388
1,Small Business Development Center,South San Diego SBDC,880 National City Blvd.,619-482-6391
2,Small Business Development Center,Export SBDC at World Trade Center San Diego,530 B St. Suite 700,None
3,Small Business Development Center,The Brink SBDC,5998 Alcala Park,619-260-4547
4,Small Business Development Center,Alliance SBDC,7675 Dagget St. Suite 340,858-277-2822
5,Small Business Development Center,East County SBDC,127 East Lexington Ave.,619-258-3670
6,Small Business Development Center,Connect SBDC,9605 Scranton Road Suite 110,858-964-1300
7,Small Business Development Center,San Diego and Imperial SBDC Network (Lead Office),880 National City Blvd,619-482-6388
8,Small Business Development Center,South San Diego SBDC,880 National City Blvd.,619-482-6391
9,Small Business Agency,SBA San Diego District,550 West C St. suite 550,619-557-7250


Geocoding all the addresses to create spatial features

In [6]:
sba_feat_list = []
for index, row in sbaDF.iterrows():
    address = row['address'] + ', San Diego County'
    print(address)
    hit = geocode(address, max_locations=1)[0]
    attributes=hit['attributes']
    attributes['org_type'] = row['org_type']
    attributes['org_name'] = row['org_name']
    attributes['orig_addr'] = row['address']
    attributes['cont_nmbr'] = row['cont_nmbr']
    
    sba_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    sba_feat_list.append(sba_feat)

880 National City Blvd, San Diego County
880 National City Blvd., San Diego County
530 B St. Suite 700, San Diego County
 5998 Alcala Park, San Diego County
7675 Dagget St. Suite 340, San Diego County
127 East Lexington Ave., San Diego County
9605 Scranton Road Suite 110, San Diego County
880 National City Blvd, San Diego County
880 National City Blvd., San Diego County
550 West C St. suite 550, San Diego County


In [7]:
sba_fset = FeatureSet(features=sba_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [8]:
sba_sdf = sba_fset.sdf
sba_sdf

,Loc_name,Status,Score,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,...,Xmax,Ymin,Ymax,ExInfo,org_type,org_name,orig_addr,cont_nmbr,OBJECTID,SHAPE
0,World,M,100,"880 National City Blvd, National City, Califor...","880 National City Blvd, National City, CA, 919...",880 National City Blvd,StreetAddress,,,"880 National City Blvd, National City, Califor...",...,-117.105795,32.674143,32.676143,,Small Business Development Center,San Diego and Imperial SBDC Network (Lead Office),880 National City Blvd,619-482-6388,1,"{""x"": -117.10679481570502, ""y"": 32.67514250187..."
1,World,M,100,"880 National City Blvd, National City, Califor...","880 National City Blvd, National City, CA, 919...",880 National City Blvd,StreetAddress,,,"880 National City Blvd, National City, Califor...",...,-117.105795,32.674143,32.676143,,Small Business Development Center,South San Diego SBDC,880 National City Blvd.,619-482-6391,2,"{""x"": -117.10679481570502, ""y"": 32.67514250187..."
2,World,M,100,"530 B St, Ste 700, San Diego, California, 92101","530 B St, Ste 700, San Diego, CA, 92101, USA","530 B St, Ste 700",Subaddress,,,"530 B St, Ste 700, San Diego, California, 92101",...,-117.15873,32.717124,32.719124,,Small Business Development Center,Export SBDC at World Trade Center San Diego,530 B St. Suite 700,<NA>,3,"{""x"": -117.15973002703848, ""y"": 32.71782752253..."
3,World,M,100,"5998 Alcala Park, San Diego, California, 92110","5998 Alcala Park, San Diego, CA, 92110, USA",5998 Alcala Park,StreetAddress,,,"5998 Alcala Park, San Diego, California, 92110",...,-117.185197,32.770941,32.772941,,Small Business Development Center,The Brink SBDC,5998 Alcala Park,619-260-4547,4,"{""x"": -117.18619703470044, ""y"": 32.77194109857..."
4,World,M,100,"7675 Dagget St, Ste 340, San Diego, California...","7675 Dagget St, Ste 340, San Diego, CA, 92111,...","7675 Dagget St, Ste 340",Subaddress,,,"7675 Dagget St, Ste 340, San Diego, California...",...,-117.157369,32.822228,32.824228,,Small Business Development Center,Alliance SBDC,7675 Dagget St. Suite 340,858-277-2822,5,"{""x"": -117.15837534384727, ""y"": 32.82353845090..."
5,World,M,100,"127 E Lexington Ave, El Cajon, California, 92020","127 E Lexington Ave, El Cajon, CA, 92020, USA",127 E Lexington Ave,PointAddress,,,"127 E Lexington Ave, El Cajon, California, 92020",...,-116.960735,32.791484,32.793484,,Small Business Development Center,East County SBDC,127 East Lexington Ave.,619-258-3670,6,"{""x"": -116.96173078239045, ""y"": 32.79270101003..."
6,World,M,100,"9605 Scranton Rd, Ste 110, San Diego, Californ...","9605 Scranton Rd, Ste 110, San Diego, CA, 9212...","9605 Scranton Rd, Ste 110",Subaddress,,,"9605 Scranton Rd, Ste 110, San Diego, Californ...",...,-117.201305,32.894705,32.896705,,Small Business Development Center,Connect SBDC,9605 Scranton Road Suite 110,858-964-1300,7,"{""x"": -117.20285248812408, ""y"": 32.89570500277..."
7,World,M,100,"880 National City Blvd, National City, Califor...","880 National City Blvd, National City, CA, 919...",880 National City Blvd,StreetAddress,,,"880 National City Blvd, National City, Califor...",...,-117.105795,32.674143,32.676143,,Small Business Development Center,San Diego and Imperial SBDC Network (Lead Office),880 National City Blvd,619-482-6388,8,"{""x"": -117.10679481570502, ""y"": 32.67514250187..."
8,World,M,100,"880 National City Blvd, National City, Califor...","880 National City Blvd, National City, CA, 919...",880 National City Blvd,StreetAddress,,,"880 National City Blvd, National City, Califor...",...,-117.105795,32.674143,32.676143,,Small Business Development Center,South San Diego SBDC,880 National City Blvd.,619-482-6391,9,"{""x"": -117.10679481570502, ""y"": 32.67514250187..."
9,World,M,100,"550 W C St, Ste 550, San Diego, California, 92101","550 W C St, Ste 550, San Diego, CA, 92101, USA","550 W C St, Ste 550",Subaddress,,,"550 W C St, Ste 550, San Diego, California, 92101",...,-117.166841,32.716346,32.718346,,Small Business Agency,SBA S

Creating the feature layer so that we have pre-computed spatial features.

In [9]:
%%time
title="San Diego County SBA Centers"
tags = ['SBA','SanDiegoCounty']
lyr = create_feature_layer(gis, sba_sdf, title, tags)
lyr

CPU times: user 130 ms, sys: 17.6 ms, total: 147 ms
Wall time: 9.95 s


<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2567d/FeatureServer/0">